In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.7.0'

In [2]:
model = keras.models.load_model('dogs_cats_10_0.687.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dogs_cats_10_0.687.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Bany\AppData\Local\Temp\tmpxnmsdkx6\assets


### Question 1

In [4]:
import os
 
file_size = os.path.getsize('dogs_cats_10_0.687.tflite')
print("File Size is :", file_size, "bytes")

File Size is : 44865956 bytes


In [5]:
import tensorflow.lite as tflite

In [6]:
interpreter = tflite.Interpreter(model_path='dogs_cats_10_0.687.tflite')
interpreter.allocate_tensors()

In [7]:
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_1_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [8]:
input_index = interpreter.get_input_details()[0]['index']
input_index

0

### Question 2

In [9]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [10]:
output_index = interpreter.get_output_details()[0]['index']
output_index

13

### Preparing the image

In [11]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [12]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg')

In [13]:
img = prepare_image(img, target_size=(150,150))

In [14]:
x = np.array(img, dtype=np.float32)
x = x /255

### Question 3

In [15]:
x[0][0][0]

0.7058824

In [16]:
X = np.array([x])

In [17]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

### Question 4

In [18]:
preds

array([[0.7704913]], dtype=float32)

### Question 6 (predict via notebook)

In [23]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg'}

result = requests.post(url, json=data).json()
print(result)

[0.5413472652435303]
